In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\d

In [2]:
import collections
import os
import cv2
import pandas as pd
import numpy as np
import random

In [4]:
labels = pd.read_csv('train.truth.csv', sep=',')

In [5]:
DATA_FOLDER = 'train'
data = []
for file in labels.fn.tolist():
    file_path = os.path.join(DATA_FOLDER, file)
    data.append(cv2.imread(file_path))
data = np.array(data)
print('Samples: ', data.shape[0],
      'x-axis width: ', data.shape[1],
      'y-axis height: ', data.shape[2],
      'colors: ', data.shape[3])

Samples:  48896 x-axis width:  64 y-axis height:  64 colors:  3


In [40]:
NOT_ROTATE = -1
NUMBER_CLASSES = 4

def rot_corrections(label_name):
    if label_name == 'rotated_left':
        return cv2.ROTATE_90_CLOCKWISE
    elif label_name == 'upright':
        return NOT_ROTATE
    elif label_name == 'rotated_right':
        return cv2.ROTATE_90_COUNTERCLOCKWISE
    elif label_name == 'upside_down':    
        return cv2.ROTATE_180

def correct_image(image, rot_correction):
    if rot_correction == NOT_ROTATE:
        return image
    else:
        return cv2.rotate(image, rot_correction)    
    

In [7]:
labels['rot_corrections'] = labels.label.apply(rot_corrections)

In [8]:
for i in range(15):
    rotated = correct_image(data[i], labels.loc[i, 'rot_corrections'])
    cv2.imwrite(str(i) + 'image.jpg', data[i])
    cv2.imwrite(str(i) + 'rotated.jpg', rotated)

In [43]:
x_train = data[34227:]
x_test = data[:34227]
y_train = np.array(labels.iloc[34227:].rot_corrections.tolist())
y_train = y_train.reshape(len(y_train), 1)
y_test = np.array(labels.iloc[:34227].rot_corrections.tolist())
y_test = y_test.reshape(len(y_test), 1)

In [44]:
y_train = keras.utils.to_categorical(y_train, NUMBER_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUMBER_CLASSES)
x_train =x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [52]:
batch_size = 32
epochs = 100
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'

In [ ]:
# MODEL DEFINITION

In [50]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUMBER_CLASSES))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
                                                                            #I am using the old version here
    
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    

# RUN MODEL

In [ ]:

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Using real-time data augmentation.
Epoch 1/100
459/459 [==============================] - 17s 38ms/step - loss: 1.2518 - acc: 0.4137 - val_loss: 0.9706 - val_acc: 0.5922
Epoch 2/100
459/459 [==============================] - 13s 28ms/step - loss: 0.9863 - acc: 0.5483 - val_loss: 0.7497 - val_acc: 0.6615
Epoch 3/100
459/459 [==============================] - 13s 27ms/step - loss: 0.8388 - acc: 0.6001 - val_loss: 0.6986 - val_acc: 0.6298
Epoch 4/100
459/459 [==============================] - 13s 28ms/step - loss: 0.7692 - acc: 0.6188 - val_loss: 0.6151 - val_acc: 0.6765
Epoch 5/100
459/459 [==============================] - 13s 28ms/step - loss: 0.7246 - acc: 0.6406 - val_loss: 0.6154 - val_acc: 0.6716
Epoch 6/100
456/459 [============================>.] - ETA: 0s - loss: 0.6980 - acc: 0.6500

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])